# 맵픽 서비스랩의 테마지도 제작을 위한 프로젝트입니다.
- 2024년 최대 이슈인 의대 정원 증원에 대해 살펴보고
- 그 영향을 시각화하여 지도에 표시하고자 합니다.


In [1]:
# 필요라이브러리
import pandas as pd

/var/folders/_p/b25fmdzs0gs_7l54_j6744sm0000gn/T/ipykernel_9006/4145177196.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# 원천데이터
- 202405_202405_연령별인구현황_월간.csv: 시도별/연령별 데이터
- BND_SIDO_PG.zip: 시도레이어

# 본 소스코드는 다음의 사항을 다루고 있습니다.
- 1. GIS 시각화를 위한 시도 데이터(shp) 가공
- 2. 인구-의대 정원 증원 전후 비교를 위한 연령별 인구데이터-시도데이터 매칭
- 3. 시도별 의대정원 비교를 위한 2.와 학교별 의대 정원 데이터 매칭

In [2]:
#시도별 레이어 데이터
import geopandas as gpd
file = gpd.read_file('../BND_SIDO_PG.zip', encoding = 'cp949')
gdf = gpd.GeoDataFrame(file, crs = 'EPSG:5186')
gdf.head()

,BASE_DATE,SIDO_CD,SIDO_NM,geometry
0,20230701,11,서울특별시,"POLYGON ((201748.532 566815.017, 201953.244 56..."
1,20230701,21,부산광역시,"MULTIPOLYGON (((378441.217 256011.416, 378440...."
2,20230701,22,대구광역시,"POLYGON ((330688.544 415299.942, 330688.134 41..."
3,20230701,23,인천광역시,"MULTIPOLYGON (((91935.674 481967.049, 91936.11..."
4,20230701,24,광주광역시,"POLYGON ((178186.330 295822.605, 178255.423 29..."


In [3]:
#시도별/연령별 인구데이터
df_pop = pd.read_csv('../202405_202405_연령별인구현황_월간.csv',
                      encoding = 'cp949',
                      thousands= ',')
#최상단의 전국단위 제거
df_pop = df_pop.iloc[1:,:]
df_pop.head()



,행정구역,2024년05월_계_총인구수,2024년05월_계_연령구간인구수,2024년05월_계_0~9세,2024년05월_계_10~19세,2024년05월_계_20~29세,2024년05월_계_30~39세,2024년05월_계_40~49세,2024년05월_계_50~59세,2024년05월_계_60~69세,...,2024년05월_여_10~19세,2024년05월_여_20~29세,2024년05월_여_30~39세,2024년05월_여_40~49세,2024년05월_여_50~59세,2024년05월_여_60~69세,2024년05월_여_70~79세,2024년05월_여_80~89세,2024년05월_여_90~99세,2024년05월_여_100세 이상
1,서울특별시 (1100000000),9371930,9371930,499875,729437,1340759,1431670,1403182,1495784,1323704,...,356752,707162,715572,709033,760407,697163,411355,206896,34635,1149
2,부산광역시 (2600000000),3282751,3282751,188948,262934,369420,389821,478626,540440,562390,...,128139,180067,188812,236724,276199,297793,179382,90104,13978,370
3,대구광역시 (2700000000),2367803,2367803,146691,212656,275253,280691,350673,419806,370680,...,103198,130357,130937,176785,214931,193446,110691,63316,10088,224
4,인천광역시 (2800000000),3009873,3009873,199369,271852,358787,417484,478415,517868,450332,...,132345,173043,199695,232415,259811,228004,108742,61434,12180,401
5,광주광역시 (2900000000),1414254,1414254,96572,146313,186719,173599,224220,242776,187319,...,70866,89182,83825,112428,122676,97614,55997,30495,5567,180


# 인구데이터와 시도 레이어 매칭
- 1. 인구데이터에서 성별현황 제거
- 2. 시도레이어-인구데이터 시도명 컬럼명 및 value 매칭
- 3. 시도명을 이용한 merge

In [4]:
gdf.columns

Index(['BASE_DATE', 'SIDO_CD', 'SIDO_NM', 'geometry'], dtype='object')

In [5]:
# 총인구수 및 성별 컬럼제거
df_pop.columns

Index(['행정구역', '2024년05월_계_총인구수', '2024년05월_계_연령구간인구수', '2024년05월_계_0~9세',
       '2024년05월_계_10~19세', '2024년05월_계_20~29세', '2024년05월_계_30~39세',
       '2024년05월_계_40~49세', '2024년05월_계_50~59세', '2024년05월_계_60~69세',
       '2024년05월_계_70~79세', '2024년05월_계_80~89세', '2024년05월_계_90~99세',
       '2024년05월_계_100세 이상', '2024년05월_남_총인구수', '2024년05월_남_연령구간인구수',
       '2024년05월_남_0~9세', '2024년05월_남_10~19세', '2024년05월_남_20~29세',
       '2024년05월_남_30~39세', '2024년05월_남_40~49세', '2024년05월_남_50~59세',
       '2024년05월_남_60~69세', '2024년05월_남_70~79세', '2024년05월_남_80~89세',
       '2024년05월_남_90~99세', '2024년05월_남_100세 이상', '2024년05월_여_총인구수',
       '2024년05월_여_연령구간인구수', '2024년05월_여_0~9세', '2024년05월_여_10~19세',
       '2024년05월_여_20~29세', '2024년05월_여_30~39세', '2024년05월_여_40~49세',
       '2024년05월_여_50~59세', '2024년05월_여_60~69세', '2024년05월_여_70~79세',
       '2024년05월_여_80~89세', '2024년05월_여_90~99세', '2024년05월_여_100세 이상'],
      dtype='object')

In [6]:
df_pop = df_pop[['행정구역', '2024년05월_계_0~9세',
       '2024년05월_계_10~19세', '2024년05월_계_20~29세', '2024년05월_계_30~39세',
       '2024년05월_계_40~49세', '2024년05월_계_50~59세', '2024년05월_계_60~69세',
       '2024년05월_계_70~79세', '2024년05월_계_80~89세', '2024년05월_계_90~99세',
       '2024년05월_계_100세 이상']]
df_pop.head()

,행정구역,2024년05월_계_0~9세,2024년05월_계_10~19세,2024년05월_계_20~29세,2024년05월_계_30~39세,2024년05월_계_40~49세,2024년05월_계_50~59세,2024년05월_계_60~69세,2024년05월_계_70~79세,2024년05월_계_80~89세,2024년05월_계_90~99세,2024년05월_계_100세 이상
1,서울특별시 (1100000000),499875,729437,1340759,1431670,1403182,1495784,1323704,750707,347948,47360,1504
2,부산광역시 (2600000000),188948,262934,369420,389821,478626,540440,562390,326516,145073,18147,436
3,대구광역시 (2700000000),146691,212656,275253,280691,350673,419806,370680,198457,99455,13180,261
4,인천광역시 (2800000000),199369,271852,358787,417484,478415,517868,450332,203112,96617,15540,497
5,광주광역시 (2900000000),96572,146313,186719,173599,224220,242776,187319,100454,48862,7217,203


In [7]:
#행정구역의 컬럼명 -> 시도명만 남기고 제거
df_pop['행정구역'] = df_pop['행정구역'].apply(lambda x: x.split('(')[0])
df_pop.head()

,행정구역,2024년05월_계_0~9세,2024년05월_계_10~19세,2024년05월_계_20~29세,2024년05월_계_30~39세,2024년05월_계_40~49세,2024년05월_계_50~59세,2024년05월_계_60~69세,2024년05월_계_70~79세,2024년05월_계_80~89세,2024년05월_계_90~99세,2024년05월_계_100세 이상
1,서울특별시,499875,729437,1340759,1431670,1403182,1495784,1323704,750707,347948,47360,1504
2,부산광역시,188948,262934,369420,389821,478626,540440,562390,326516,145073,18147,436
3,대구광역시,146691,212656,275253,280691,350673,419806,370680,198457,99455,13180,261
4,인천광역시,199369,271852,358787,417484,478415,517868,450332,203112,96617,15540,497
5,광주광역시,96572,146313,186719,173599,224220,242776,187319,100454,48862,7217,203


In [8]:
# df_pop의 행정구역 공백 제거
df_pop['행정구역'] = df_pop['행정구역'].str.strip('   ')
df_pop.rename({'행정구역':'시도명'}, axis = 1, inplace = True)

In [9]:
gdf = gdf[['SIDO_NM', 'geometry']]
gdf.rename({'SIDO_NM':'시도명'}, axis = 1, inplace = True)

In [10]:
print(df_pop.columns)

Index(['시도명', '2024년05월_계_0~9세', '2024년05월_계_10~19세', '2024년05월_계_20~29세',
       '2024년05월_계_30~39세', '2024년05월_계_40~49세', '2024년05월_계_50~59세',
       '2024년05월_계_60~69세', '2024년05월_계_70~79세', '2024년05월_계_80~89세',
       '2024년05월_계_90~99세', '2024년05월_계_100세 이상'],
      dtype='object')


In [11]:
print(gdf.columns)

Index(['시도명', 'geometry'], dtype='object')


In [17]:
gdf['시도명'] = gdf['시도명'].apply( lambda x: '전북특별자치도' if x == '전라북도' else x)

In [18]:
for i, j in zip(sorted(list(df_pop['시도명'])),sorted(list(gdf['시도명']))):
    print(i, ':', j)

강원특별자치도 : 강원특별자치도
경기도 : 경기도
경상남도 : 경상남도
경상북도 : 경상북도
광주광역시 : 광주광역시
대구광역시 : 대구광역시
대전광역시 : 대전광역시
부산광역시 : 부산광역시
서울특별시 : 서울특별시
세종특별자치시 : 세종특별자치시
울산광역시 : 울산광역시
인천광역시 : 인천광역시
전라남도 : 전라남도
전북특별자치도 : 전북특별자치도
제주특별자치도 : 제주특별자치도
충청남도 : 충청남도
충청북도 : 충청북도


# merge
- 1. 시도 레이어-연령별 인구
- 2. 시도-의대 정원

In [19]:
sido_pop = df_pop.merge(gdf, on = '시도명', how = 'left')
sido_pop

,시도명,2024년05월_계_0~9세,2024년05월_계_10~19세,2024년05월_계_20~29세,2024년05월_계_30~39세,2024년05월_계_40~49세,2024년05월_계_50~59세,2024년05월_계_60~69세,2024년05월_계_70~79세,2024년05월_계_80~89세,2024년05월_계_90~99세,2024년05월_계_100세 이상,geometry
0,서울특별시,499875,729437,1340759,1431670,1403182,1495784,1323704,750707,347948,47360,1504,"POLYGON ((201748.532 566815.017, 201953.244 56..."
1,부산광역시,188948,262934,369420,389821,478626,540440,562390,326516,145073,18147,436,"MULTIPOLYGON (((378441.217 256011.416, 378440...."
2,대구광역시,146691,212656,275253,280691,350673,419806,370680,198457,99455,13180,261,"POLYGON ((330688.544 415299.942, 330688.134 41..."
3,인천광역시,199369,271852,358787,417484,478415,517868,450332,203112,96617,15540,497,"MULTIPOLYGON (((91935.674 481967.049, 91936.11..."
4,광주광역시,96572,146313,186719,173599,224220,242776,187319,100454,48862,7217,203,"POLYGON ((178186.330 295822.605, 178255.423 29..."
5,대전광역시,94268,136971,197113,188850,216620,244497,203974,101423,49705,7480,218,"POLYGON ((235744.989 432391.437, 235851.457 43..."
6,울산광역시,76310,109084,119947,136450,175851,204284,172245,73891,28585,3870,92,"MULTIPOLYGON (((410247.660 306050.069, 410246...."
7,세종특별자치시,42556,51264,37201,59072,77119,55745,37887,16937,8373,1580,40,"POLYGON ((214835.355 459451.226, 214843.683 45..."
8,경기도,966673,1318138,1656250,1887017,2235909,2352886,1869159,868065,432273,66404,1936,"MULTIPOLYGON (((185484.662 483430.400, 185529...."
9,강원특별자치도,88617,127804,159949,158095,204559,262499,277871,144359,85644,13135,420,"MULTIPOLYGON (((410017.021 508090.999, 410009...."


In [20]:
sido_pop.columns = ['sido_nm', 'under_10', 'age_10', 'age_20',
       'age_30', 'age_40', 'age_50',
       'age_60', 'age_70', 'age_80',
       'age_90', 'age_100', 'geometry']

In [21]:
gdf_sido_pop = gpd.GeoDataFrame(sido_pop,
                            crs = file.crs ,
                            geometry = 'geometry')
gdf_sido_pop.to_file('../시도_연령별_인구', encoding = 'cp949')